In [2]:
import csv
import pickle
import decimal  # 이것은 시간이 오래 걸리기 때문에 주문이 들어기지 않는 경우에만 사용하는 것이 좋다(전송 수량 찾을 때 굳)

# ck
import datetime
# import pytictoc
import sys

sys.setrecursionlimit(1000000)  # 재귀함수 제한 늘리기
import pandas as pd
import numpy as np
import pip
import sys
import requests
import json
# api json 파싱 python 업비트
import jwt  # PyJWT
import uuid
import requests
from urllib.parse import urlencode
import pyupbit  # 업비트#버젼 0.2.8
print(pyupbit.Upbit)
import ccxt  # 바이낸스
print(ccxt)


import pprint
import time
import math

import binance #!pip install binance-connector
import os
import hashlib
from binance.client import Client # python-binance
########선물거래를 위해#######
from binance_f import RequestClient
from binance_f.constant.test import *
from binance_f.base.printobject import *
from binance_f.model.constant import *
from IPython.display import display, clear_output
from binance.spot import Spot #!pip install binance-connector

##############################소수점 정리....##############################################

##@@@ 이 두개도 합치기


###########업비트에서 환율 조회


headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


def upbit_get_usd_krw():
    try:
        url = 'https://quotation-api-cdn.dunamu.com/v1/forex/recent?codes=FRX.KRWUSD'
        exchange = requests.get(url, headers=headers).json()
        return exchange[0]['basePrice']
    except:
        pass


# https://docs.upbit.com/docs/market-info-trade-price-detail
################################원화 마켄 주문 최소 주문 가격 단위###########################
def UP_Minimum_Order_cash_Size(up_pre):  ##@@여기 전부 가격을 넣는 형식으로 바꿈.
    #     up_pre = pyupbit.get_current_price("KRW-" + symbol)  # 해당코인 가격조회
    # print(up_pre,"원화 최소 주문 가격 단위 조회")
    if (2000000 <= up_pre):
        return 1000
    if (1000000 <= up_pre < 2000000):
        return 500
    if (500000 <= up_pre < 1000000):
        return 100
    if (100000 <= up_pre < 500000):
        return 50
    if (10000 <= up_pre < 100000):
        return 10
    if (1000 <= up_pre < 10000):
        return 5
    if (100 <= up_pre < 1000):
        return 1
    if (10 <= up_pre < 100):
        return 0.1
    if (0 <= up_pre < 10):
        return 0.01


#########################################전송가능 심볼명#########################################
######심볼명 예외처리로 중복제거를 위해######
def removeAllOccur(l, i):
    try:
        while True: l.remove(i)
    except ValueError:
        pass


#########################################
def bin_fu_setting(send_to_bin_optimal_symbol, future_leverage):
    try:
        result = request_client.change_margin_type(symbol=send_to_bin_optimal_symbol + "USDT", marginType=FuturesMarginType.ISOLATED)
    #         PrintBasic.print_obj(result)
    except:#         print("마진타입 변화 불필요")
        pass
    
    try:
        result = request_client.change_initial_leverage(symbol=send_to_bin_optimal_symbol + "USDT", leverage=future_leverage)
    #         PrintBasic.print_obj(result) # 레버리지 변화 (자본에 따라 기본 10배)
    except:
        pass
    
    


#     PrintBasic.print_obj(result)

def up_spot_ask_bid_info(sym_name):
    # 업비트 매수, 매도 호가창 한번에 조회
    up_sym_names = ["KRW-" + sym_name[i] for i in range(len(sym_name))]
    up_spot_ask_bid = pyupbit.get_orderbook(ticker=up_sym_names)
    return up_sym_names, up_spot_ask_bid


def up_ask_info(up_sym_names, up_spot_ask_bid):  # 업비트 매수 호가창
    up_ask_price = []
    up_ask_size = []
    up_ask_price_append = up_ask_price.append
    up_ask_size_append = up_ask_size.append
    for i in range(len(up_sym_names)):
        up_ask_price_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['ask_price']))
        up_ask_size_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['ask_size']))
    #         print(up_ask_price[i],up_ask_size[i],up_sym_names[i])
    return up_ask_price, up_ask_size


def up_bid_info(up_sym_names, up_spot_ask_bid):  # 업비트 매도 호가창
    up_bid_price = []
    up_bid_size = []
    up_bid_price_append = up_bid_price.append
    up_bid_size_append = up_bid_size.append
    for i in range(len(up_sym_names)):
        up_bid_price_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['bid_price']))
        up_bid_size_append(float(next((item for item in up_spot_ask_bid if item['market'] == up_sym_names[i]), None)['orderbook_units'][0]['bid_size']))
    #         print(up_bid_price[i],up_bid_size[i],up_sym_names[i])
    return up_bid_price, up_bid_size

# up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info(sym_name)
# up_ask_price, up_ask_size = up_ask_info(up_sym_names,up_spot_ask_bid)
# up_bid_price, up_bid_size= up_bid_info(up_sym_names,up_spot_ask_bid)
def bin_spot_ask_bid_info(sym_name):  # 바이낸스 현물 호가창 한번에 조회
    bin_sym_names = [sym_name[i] + "USDT" for i in range(len(sym_name))]
    bin_spot_ask_bid = client_spot.book_ticker()
    return bin_sym_names, bin_spot_ask_bid


def bin_ask_info(bin_sym_names, bin_spot_ask_bid):  # 바이낸스 현물 호가창 조회로 매수 호가창 가져옴
    bin_spot_ask_price = []
    bin_spot_ask_size = []
    bin_spot_ask_price_append = bin_spot_ask_price.append
    bin_spot_ask_size_append = bin_spot_ask_size.append
    for i in range(len(bin_sym_names)):
        bin_spot_ask_price_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askPrice']))
        bin_spot_ask_size_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askQty']))
    #         print(bin_spot_ask_price[i],bin_spot_ask_size[i],bin_sym_names[i])
    return bin_spot_ask_price, bin_spot_ask_size


def bin_bid_info(bin_sym_names, bin_spot_ask_bid):  # 바이낸스 현물 호가창 조회로 매도 호가창 가져옴
    bin_spot_bid_price = []
    bin_spot_bid_size = []
    bin_spot_bid_price_append = bin_spot_bid_price.append
    bin_spot_bid_size_append = bin_spot_bid_size.append
    
    for i in range(len(bin_sym_names)):
        bin_spot_bid_price_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidPrice']))
        bin_spot_bid_size_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidQty']))
    #         print(bin_spot_bid_price[i],bin_spot_bid_size[i],bin_sym_names[i])
    return bin_spot_bid_price, bin_spot_bid_size

# bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name)
# bin_spot_ask_price,bin_spot_ask_size = bin_ask_info(bin_sym_names,bin_spot_ask_bid)
# bin_spot_bid_price,bin_spot_bid_size = bin_bid_info(bin_sym_names,bin_spot_ask_bid)

def bin_fu_ask_bid_info(sym_name):
    bin_sym_names = [sym_name[i] + "USDT" for i in range(len(sym_name))]
    bin_fu_ask_bid = client.futures_orderbook_ticker()

    return bin_sym_names, bin_fu_ask_bid


def bin_fu_ask_info(bin_sym_names, bin_fu_ask_bid):
    bin_fu_ask_price = []
    bin_fu_ask_size = []
    bin_fu_ask_price_append = bin_fu_ask_price.append
    bin_fu_ask_size_append = bin_fu_ask_size.append
    for i in range(len(bin_sym_names)):
        bin_fu_ask_price_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askPrice']))
        bin_fu_ask_size_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askQty']))
    #         print(bin_fu_ask_price[i],bin_fu_ask_size[i],bin_sym_names[i])
    return bin_fu_ask_price, bin_fu_ask_size


def bin_fu_bid_info(bin_sym_names, bin_fu_ask_bid):
    bin_fu_bid_price = []
    bin_fu_bid_size = []
    bin_fu_bid_price_append = bin_fu_bid_price.append
    bin_fu_bid_size_append = bin_fu_bid_size.append
        
    for i in range(len(bin_sym_names)):
        bin_fu_bid_price_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidPrice']))
        bin_fu_bid_size_append(float(next((item for item in bin_fu_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidQty']))
    #         print(bin_fu_bid_price[i],bin_fu_bid_size[i],bin_sym_names[i])
    return bin_fu_bid_price, bin_fu_bid_size

# bin_sym_names,bin_fu_ask_bid = bin_fu_ask_bid_info(sym_name)
# bin_fu_ask_price,bin_fu_ask_size = bin_fu_ask_info(sym_name,bin_fu_ask_bid)
# bin_fu_bid_price,bin_fu_bid_size = bin_fu_bid_info(sym_name,bin_fu_ask_bid)

def find_max_per(second_using_asset, sym_name, bin_spot_ask_price, bin_withdrawfee):
    max_per = []
    max_per_append = max_per.append
    for i in range(len(sym_name)):
        max_per_append(bin_spot_ask_price[i] * bin_withdrawfee[i] / ((second_using_asset / Usdt) / 2))

    return max_per[max_per.index(max(max_per))]


def Adjusting_second_using_asset(second_using_asset, up_market_fee, bin_spot_market_fee, bin_future_market_fee, per, adjust_leverage):
    
    #레버리지를 비율 조정.. 이 부분 맞나?
#     if (adjust_leverage == 1):
#         pass
#     else:
#         adjust_leverage = adjust_leverage - 1
        
    a = (1 - bin_future_market_fee) * adjust_leverage # 바
    b = (1 - bin_spot_market_fee) * (1 - up_market_fee) * (1 - per)  # *(1-0.0004)#@ 0.0004는 추가 보정치 #업

    c = a / (a + b)  # 업
    d = b / (a + b)  # 바

    bin_spot_Usdt_Send_to_up = second_using_asset * c
    bin_fu_Usdt_asset = second_using_asset * d

    return bin_spot_Usdt_Send_to_up, bin_fu_Usdt_asset


# def Adjusting_first_using_asset(sym_name,up_bid_price,up_withdrawfee,up_asset,binance_asset,up_market_fee,bin_spot_market_fee,bin_future_market_fee):
#     max_per = []
#     for i in range(len(sym_name)):
#         max_per.append(up_bid_price[i]* up_withdrawfee[i])
#     max_val= max_per[max_per.index(max(max_per))]

#     a = (1- up_market_fee) * (1-(max_val/up_asset)) *(1- bin_spot_market_fee)
#     b = (1- bin_future_market_fee)

#     c = (a / (a + b)) * (up_asset + binance_asset)
#     d = (b / (a + b)) * (up_asset + binance_asset)
# #     print(c,d)
#     return c,d #바낸자본 업비트 자본

def up_to_bin_canwithdrow_fee(sym_name):
    server_url = 'https://api.upbit.com'
    query = {'currency': sym_name, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
    # 해당 코인 전송수수료(해당코인 시가 * 전송 수수료)
    symbol_fee = float(res.json()['currency']['withdraw_fee'])
    #         print(symbol_fee)
    #     upbit_withdraw_fee = float(res.json()['currency']['withdraw_fee']) * up_pre
    #         print(upbit_withdraw_fee,"업비트 보낼 시 심볼 전송 가격")

    bin_sym_send_state = res.json()['withdraw_limit']['can_withdraw']  # 출금지원가능여부

    if (bin_sym_send_state == True):
        pass
    else:
        print(sym_name, "해당코인 업비트에서 바이낸스로 출금 불가상태")

    return symbol_fee, bin_sym_send_state


def up_deposit_address(sym_name):
    server_url = 'https://api.upbit.com'
    ############추가 ... 입출금 상태는 주소로 받아야댐..
    try:
        query = {'currency': sym_name, }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {'access_key': access_key,
                   'nonce': str(uuid.uuid4()),
                   'query_hash': query_hash,
                   'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.post(server_url + "/v1/deposits/generate_coin_address", params=query, headers=headers)
        up_add = res.json()['deposit_address']
        bin_sym_send_state_2 = True
        up_seadd = res.json()['secondary_address']

        return up_add, bin_sym_send_state_2, up_seadd
    # print(up_add, up_seadd)

    except KeyError:
        up_add = "해당코인 바이낸스에서 업비트 입금 불가상태"
        up_seadd = "해당코인 바이낸스에서 업비트 입금 불가상태"
        bin_sym_send_state_2 = False
        print(sym_name, "해당코인 바이낸스에서 업비트 입금 불가상태")
        return up_add, bin_sym_send_state_2, up_seadd


def up_deposit_address_2(sym_name):
    #### 업비트 해당코인 입금 주소 생성요청
    try:
        query = {'currency': sym_name, }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {
            'access_key': access_key,
            'nonce': str(uuid.uuid4()),
            'query_hash': query_hash,
            'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.post(server_url + "/v1/deposits/generate_coin_address", params=query,
                            headers=headers)
        up_add = res.json()['deposit_address']
        up_sym_send_state = True
        up_seadd = res.json()['secondary_address']
        # print(up_add, up_seadd)
        return up_add, up_sym_send_state, up_seadd

    except KeyError:
        up_add = "해당코인 바이낸스에서 업비트 입금 불가상태"
        up_seadd = "해당코인 바이낸스에서 업비트 입금 불가상태"
        up_sym_send_state = False
        print(sym_name, "해당코인 바이낸스에서 업비트 입금 불가상태")
        return up_add, up_sym_send_state, up_seadd


def up_bin_withdraw_fee(sym_name, all_coin_info):
    # 심볼명,달러 등은 전부 수수료를 저장해두고 나중에 갱신.
    up_withdrawfee = []
    bin_withdrawfee = []
    up_withdrawfee_append = up_withdrawfee.append
    bin_withdrawfee_append = bin_withdrawfee.append
#     all_coin_info = client_spot.coin_info()
    for i in range(len(sym_name)):
        aa = next((item for item in all_coin_info if item['coin'] == sym_name[i]), None)['networkList']
        bin_withdrawfee_append(float(next((item for item in aa if item['network'] == sym_name[i]), None)['withdrawFee']))
        symbol_fee, bin_sym_send_state = up_to_bin_canwithdrow_fee(sym_name[i])
        up_withdrawfee_append(symbol_fee)
        time.sleep(0.4)
    return up_withdrawfee, bin_withdrawfee


def bin_withdraw_fee(all_coin_info, sym_name):
    aa = next((item for item in all_coin_info if item['coin'] == sym_name), None)['networkList']
    bin_withdrawfee = (float(next((item for item in aa if item['network'] == sym_name), None)['withdrawFee']))
    return bin_withdrawfee


def sym_intersection():
    ######바이낸스 달러 선물가능 심볼명#####
    bin_fu_sym = client.futures_symbol_ticker()
    for i in range(len(bin_fu_sym)):
        if (bin_fu_sym[i]['symbol'][-4:] == 'USDT'):
            bin_fu_sym[i] = bin_fu_sym[i]['symbol'][:-4]
        else:
            bin_fu_sym[i] = 'remove'
    removeAllOccur(bin_fu_sym, 'remove')
    ########################################
    #####바이낸스 달러 현물가능 코인 심볼명#USDT만 찾기 #####
    bin_sym = client.get_symbol_ticker()

    for i in range(len(bin_sym)):
        if (bin_sym[i]['symbol'][-4:] == 'USDT'):
            bin_sym[i] = bin_sym[i]['symbol'][:-4]
        else:
            bin_sym[i] = 'remove'
    removeAllOccur(bin_sym, 'remove')
    ########################################################
    ######업비트 코인 심볼명########
    up_sym = pyupbit.get_tickers(fiat="KRW")
    for i in range(len(up_sym)):
        # tickers[i].replace('KRW-','')
        up_sym[i] = up_sym[i][4:]
    #################################
    ######심볼명 교집합#####
    # sym_name = list(set(bin_sym) & set(up_sym))  # @
    sym_name = list(set(bin_fu_sym) & set(bin_sym) & set(up_sym))
    print(len(sym_name), len(sym_name))  # 선물없이 업비트만해선..
    return sym_name


def bin_coin_withdrow_fee(all_coin_info, sym_name):
    try:
        aa = next((item for item in all_coin_info if item['coin'] == sym_name), None)['networkList']
        withdrawfee = float(next((item for item in aa if item['network'] == sym_name), None)['withdrawFee'])
        return withdrawfee
    except:
        withdrawfee = "None_network"
        return withdrawfee
        pass


def bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name):
    try: #자기자신의 네트워크를 가지고, 현재 바이낸스의 입출금 가능상태를 가져옴
        aa = next((item for item in all_coin_info if item['coin'] == sym_name), None)['networkList']
        bb = next((item for item in aa if item['network'] == sym_name), None)
        depositEnable, withdrawEnable = bb['depositEnable'] , bb['withdrawEnable']
    except:
        depositEnable, withdrawEnable = False ,False

    return depositEnable, withdrawEnable


def del_bin_disable_withdrow_deopsit(all_coin_info, sym_name):
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):  # 바이낸스에서 입금 가능 , 전송 불가능
        depositEnable, withdrawEnable = bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name[i])
        #         print(sym_name[i],depositEnable,withdrawEnable)
        if (depositEnable == False or withdrawEnable == False):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name


def delet_None_network(all_coin_info, sym_name):
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):  # 바이낸스에서 전송
        withdrawfee = bin_coin_withdrow_fee(all_coin_info, sym_name[i])
        #         print(sym_name[i],withdrawfee)
        if (withdrawfee == "None_network"):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name



def regester_by_upbit_withdrow_addres(sym_name):
    for i in range(len(sym_name)):
#         print(sym_name[i])
        try:
            bin_address = client_spot.deposit_address(sym_name[i])['address']
            bin_Memo = client_spot.deposit_address(sym_name[i])['tag']
            time.sleep(0.1)
            print(sym_name[i])
            print('bin_address', bin_address)
            print('bin_Memo', bin_Memo)
        except:
            print(sym_name[i],"바이낸스 지갑이 닫혔을 수도 있음")


# def Adjusting_second_using_asset(second_using_asset,up_market_fee,bin_spot_market_fee,bin_future_market_fee):
#     a = (1- bin_future_market_fee) #바
#     b = (1- bin_spot_market_fee) * (1- up_market_fee) #업

#     c = a / (a+b) #업
#     d = b / (a+b) #바

#     bin_spot_Usdt_Send_to_up = second_using_asset * c
#     bin_fu_Usdt_asset = second_using_asset * d
#     return bin_spot_Usdt_Send_to_up , bin_fu_Usdt_asset

def Bin_Minimum_Spot_Order_Size(sym_name):
    Bin_Spot_Minimum_Order_Size = []
    Bin_Spot_Minimum_Order_Size_append = Bin_Spot_Minimum_Order_Size.append
    aa = client.get_exchange_info()

    for i in range(len(sym_name)):
        bb = next((item for item in aa['symbols'] if item['symbol'] == sym_name[i] + 'USDT'), None)
        Bin_Spot_Minimum_Order_Size_append(float(next((item for item in bb['filters'] if item['filterType'] == 'LOT_SIZE'), None)['minQty']))
    #     print(Bin_Spot_Minimum_Order_Size)
    return Bin_Spot_Minimum_Order_Size


# def Bin_Minimum_FU_Order_Size(sym_name):
#     Bin_Fu_Minimum_Order_Size = []
#     Bin_Fu_Minimum_Order_Size_append = Bin_Fu_Minimum_Order_Size.append
#     aa = client.futures_exchange_info()

#     for i in range(len(sym_name)):
#         bb = next((item for item in aa['symbols'] if item['symbol'] == sym_name[i] + 'USDT'), None)
#         Bin_Fu_Minimum_Order_Size_append(float(next((item for item in bb['filters'] if item['filterType'] == 'PERCENT_PRICE'), None)['multiplierDecimal']))
#     #     print(Bin_Fu_Minimum_Order_Size)
#     return Bin_Fu_Minimum_Order_Size

def Bin_Minimum_FU_Order_Size(sym_name):
    Bin_Fu_Minimum_Order_Size = []
    Bin_Fu_Minimum_Order_Size_append = Bin_Fu_Minimum_Order_Size.append
    aa = client.futures_exchange_info()
    
    for i in range(len(sym_name)):
        bb = next((item for item in aa['symbols'] if item['symbol'] == sym_name[i] + 'USDT'), None)
        Bin_Fu_Minimum_Order_Size_append(float(next((item for item in bb['filters'] if item['filterType'] == 'MARKET_LOT_SIZE'), None)['minQty']))
    #     print(Bin_Fu_Minimum_Order_Size)
    return Bin_Fu_Minimum_Order_Size


def Bin_Minimum_Withdraw_Size(symbol,all_coin_info):  # 바이낸스 최소전송가능량
    aa = next((item for item in all_coin_info if item['coin'] == symbol), None)['networkList']
    minWithdrawAmount = float(next((item for item in aa if item['network'] == symbol), None)['withdrawIntegerMultiple'])
    return minWithdrawAmount


def del_bin_disable_deopsit(all_coin_info, sym_name):  # 바이낸스 입금 불가능 코인 삭제
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):
        depositEnable, withdrawEnable = bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name[i])
        #         print(sym_name[i],depositEnable,withdrawEnable)
        if (depositEnable == False):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name


def del_bin_disable_withdraw(all_coin_info, sym_name):  # 바이낸스 출금 불가능 심볼 삭제
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):
        depositEnable, withdrawEnable = bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name[i])
        #         print(sym_name[i],depositEnable,withdrawEnable)
        if (withdrawEnable == False):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name


def del_up_disable_deopsit(sym_name):  # 업비트 입금 불가능 코인 삭제
    server_url = 'https://api.upbit.com'
    delet_list = []
    for i in range(len(sym_name)):

        query = {'currency': sym_name[i], }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash, 'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
        up_sym_whthdraw_state = res.json()['currency']['wallet_support']

        if 'deposit' in up_sym_whthdraw_state:
            pass
        else:
            delet_list.append(sym_name[i])
            print(sym_name[i], "해당코인 업비트에서 바이낸스로 입금 불가상태")

        time.sleep(0.4)
    name = list(set(sym_name) - set(delet_list))
    return name

def del_up_disable_whthdraw(sym_name):  # 업비트 출금 불가능 코인 삭제
    server_url = 'https://api.upbit.com'
    delet_list = []
    for i in range(len(sym_name)):

        query = {'currency': sym_name[i], }
        query_string = urlencode(query).encode()

        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash, 'query_hash_alg': 'SHA512', }

        jwt_token = jwt.encode(payload, secret_key)
        authorize_token = 'Bearer {}'.format(jwt_token)
        headers = {"Authorization": authorize_token}

        res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
        up_sym_whthdraw_state = res.json()['currency']['wallet_support']

        if 'withdraw' in up_sym_whthdraw_state:
            pass
        else:
            delet_list.append(sym_name[i])
            print(sym_name[i], "해당코인 업비트에서 바이낸스로 출금 불가상태")

        time.sleep(0.4)
    name = list(set(sym_name) - set(delet_list))
    return name


#####업비트 => 바이낸스 전송가능정보와 코인 수수료 가져오는코드#########################
def Up_Minimum_Withdraw_Size(sym_name):  # 업비트 최소전송가능량
    server_url = 'https://api.upbit.com'
    query = {'currency': sym_name, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)
    
    return float(res.json()['withdraw_limit']['fixed']) #이부분 틀려있었음
#     return (0.1)**float(res.json()['withdraw_limit']['fixed'])
    #         print(bin_sym_send_state,sym_name[i],"출금지원가능여부")


def up_bin_withdraw_Size(sym_name, all_coin_info):  # 바이낸스 최소전송가능량
    up_Min_Withdraw_Size = []
    bin_Min_Withdraw_Size = []
    up_Min_Withdraw_Size_append = up_Min_Withdraw_Size.append
    bin_Min_Withdraw_Size_append = bin_Min_Withdraw_Size.append
    for i in range(len(sym_name)):  # 잘되나 확인 중 나중에 지우기
        up_Min_Withdraw_Size_append(Up_Minimum_Withdraw_Size(sym_name[i]))
        bin_Min_Withdraw_Size_append(Bin_Minimum_Withdraw_Size(sym_name[i],all_coin_info))
        time.sleep(0.4)
    return up_Min_Withdraw_Size, bin_Min_Withdraw_Size


def remove_sym_name_1(all_coin_info, sym_name, Usdt, up_asset, per, ban_list, up_market_fee):  # 심볼명 리스트, 달러 환율, 자본대비 최소 퍼센테이지

    up_withdrawfee, bin_withdrawfee = up_bin_withdraw_fee(sym_name, all_coin_info)  # 업비트 전송 수수료 , 바이낸스 전송 수수료
    up_Min_Withdraw_Size, bin_Min_Withdraw_Size = up_bin_withdraw_Size(sym_name, all_coin_info)  # 업비트, 바이낸스 전송최소 가능량
    # Bin_Fu_Minimum_Order_Size = Bin_Minimum_FU_Order_Size(sym_name)  # 바이낸스 선물 최소 주문 가능량
    # Bin_Spot_Minimum_Order_Size = Bin_Minimum_Spot_Order_Size(sym_name)  # 바이낸스 현물 최소 주문가능량
    # 자본대비 수수료 , 잡다한 코인들 제외

    up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info(sym_name)
    up_ask_price, up_ask_size = up_ask_info(up_sym_names, up_spot_ask_bid)  # 업비트 매수 호가창 4
    # up_bid_price, up_bid_size = up_bid_info(up_sym_names, up_spot_ask_bid)  # 업비트 매도 호가창 1

    #     bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name)
    #     bin_spot_ask_price, bin_spot_ask_size = bin_ask_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매수 호가창 2
    #     bin_spot_bid_price, bin_spot_bid_size = bin_bid_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매도 호가창 3

    # up_to_bin_pre_rate = []
    # bin_to_up_pre_rate = []
    # up_withdrawfee_by_asset = []
    # bin_withdrawfee_by_asset = []

    for i in range(len(sym_name)):

        up_can_order_amout = up_asset * (1 - up_market_fee)  # *0.005이미함
        up_can_order_amout = up_can_order_amout + (up_withdrawfee[i] * up_ask_price[i])  ##완벽한 헷징을 위해 업비트 전송료만큼 추가..#@@이부분..계산
        Decimal_rounding = UP_Minimum_Order_cash_Size(up_ask_price[i])
        up_can_order_amout = (up_can_order_amout // Decimal_rounding) * Decimal_rounding

        can_send_Decimal = up_Min_Withdraw_Size[i]  # 전송가능 최소량@@이게 잇어야 계산 가능
        if (can_send_Decimal == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
            pass
        else:
            Decimal_rounding, Decimal_rounding_dig = (0.1) ** int(can_send_Decimal + 1), int(can_send_Decimal)  # 소수점 라운딩
            up_can_order_amout = (up_can_order_amout // Decimal_rounding) * Decimal_rounding
            up_can_order_amout = round(up_can_order_amout, Decimal_rounding_dig)

        if (((up_ask_price[i] * up_withdrawfee[i]) / up_can_order_amout) < per):  # 구매가능 정보로 해서 더 깔끔(소수점 절단)
            pass
        else:
            print((up_ask_price[i] * up_withdrawfee[i]) / up_can_order_amout, sym_name[i], "자본대비 수수료가 큰 코인")
            print(up_ask_price[i] * up_withdrawfee[i])
            sym_name[i] = 'remove'

        ###@@@ BNB코인 배제 이 부분 로직으로 대체 가능
        if ((sym_name[i] in ban_list) == True):
            sym_name[i] = 'remove'

    removeAllOccur(sym_name, 'remove')
    print(sym_name, len(sym_name))
    return sym_name


def remove_sym_name_2(all_coin_info, sym_name, Usdt, binance_asset, per, ban_list, bin_spot_market_fee):  # 심볼명 리스트, 달러 환율, 자본대비 최소 퍼센테이지

    up_withdrawfee, bin_withdrawfee = up_bin_withdraw_fee(sym_name, all_coin_info)  # 업비트 전송 수수료 , 바이낸스 전송 수수료
    up_Min_Withdraw_Size, bin_Min_Withdraw_Size = up_bin_withdraw_Size(sym_name, all_coin_info)  # 업비트, 바이낸스 전송최소 가능량
    # Bin_Fu_Minimum_Order_Size = Bin_Minimum_FU_Order_Size(sym_name)  # 바이낸스 선물 최소 주문 가능량
    Bin_Spot_Minimum_Order_Size = Bin_Minimum_Spot_Order_Size(sym_name)  # 바이낸스 현물 최소 주문가능량
    # 자본대비 수수료 , 잡다한 코인들 제외

    # up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info(sym_name)
    # up_ask_price, up_ask_size = up_ask_info(up_sym_names, up_spot_ask_bid)  # 업비트 매수 호가창 4
    # up_bid_price, up_bid_size = up_bid_info(up_sym_names, up_spot_ask_bid)  # 업비트 매도 호가창 1

    bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info(sym_name)
    bin_spot_ask_price, bin_spot_ask_size = bin_ask_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매수 호가창 2
    # bin_spot_bid_price, bin_spot_bid_size = bin_bid_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매도 호가창 3

    for i in range(len(sym_name)):

        bin_spot_Usdt_Send = binance_asset - binance_asset * bin_spot_market_fee  # 바이낸스 현물 시장가 수수료
        send_to_up_quantity = (bin_spot_Usdt_Send / bin_spot_ask_price[i])  # 바이낸스 업비트로 보낼 달러 / 티커 가격 = 수량
        send_to_up_quantity = send_to_up_quantity + bin_withdrawfee[i]
        # 전송가능 최소량과 주문가능 최소량으로 잘 자름.
        Decimal_rounding = bin_Min_Withdraw_Size[i]
        if (Decimal_rounding == 0):  # 전송 수수료가 0 인경우 나눌 수가 없음
            pass
        else:
            send_to_up_quantity = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding

        Decimal_rounding = Bin_Spot_Minimum_Order_Size[i]
        bin_spot_to_up_can_order_amount = (send_to_up_quantity // Decimal_rounding) * Decimal_rounding
        bin_spot_to_up_can_order_amount = round(bin_spot_to_up_can_order_amount, 6)
        print(sym_name[i], bin_spot_to_up_can_order_amount, (bin_spot_ask_price[i] * bin_withdrawfee[i]) / (binance_asset), bin_spot_ask_price[i])

        if (((bin_spot_ask_price[i] * bin_withdrawfee[i]) / (binance_asset)) < per):  # 구매가능 정보로 해서 더 깔끔(소수점 절단)
            #             print((up_bid_price[i] * up_withdrawfee[i]) / up_can_order_amout, sym_name[i])
            pass

        else:
            print((bin_spot_ask_price[i] * bin_withdrawfee[i]) / (binance_asset), "자본대비 수수료가 큰 코인")
            print(bin_spot_ask_price[i] * bin_withdrawfee[i])
            sym_name[i] = 'remove'

        ###@@@ BNB코인 배제 이 부분 로직으로 대체 가능
        if ((sym_name[i] in ban_list) == True):
            sym_name[i] = 'remove'

    removeAllOccur(sym_name, 'remove')
    print(sym_name, len(sym_name))
    return sym_name

# 업비트 매수 평단가
def up_avg_buy_price(sym_name):
    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()), }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)
    aa = float(next((item for item in res.json() if item['currency'] == sym_name), None)['avg_buy_price'])
    bb = float(next((item for item in res.json() if item['currency'] == sym_name), None)["balance"])
    return aa, bb


# def bin_fu_avg_buy_price(sym_name):
#     aa = float(next((item for item in client.futures_account()['positions'] if item['symbol'] == sym_name + 'USDT'), None)['entryPrice'])
#     bb = -(float(next((item for item in client.futures_account()['positions'] if item['symbol'] == sym_name + 'USDT'), None)["positionAmt"]))
#     return aa, bb
# @@ 밑에게 더 낫 지 않나 왜 안대
def bin_fu_avg_buy_price(sym_name, bin_fu_position_info):
    aa = float(next((item for item in bin_fu_position_info['positions'] if item['symbol'] == sym_name + 'USDT'), None)['entryPrice'])
    bb = -(float(next((item for item in bin_fu_position_info['positions'] if item['symbol'] == sym_name + 'USDT'), None)["positionAmt"]))
    return aa, bb


def up_withdrow_request(send_to_bin_optimal_simbol, point_of_Withdrow_UP_amount_1, bin_address, bin_Memo):
    # @ 2.코인 출금 요청#########################
    # point_of_Withdrow_UP_amount_1 조심 가져오든지 해야함
    query = {
        'currency': send_to_bin_optimal_simbol,
        'amount': point_of_Withdrow_UP_amount_1,
        'address': bin_address,
        'secondary_address': bin_Memo,
        'transaction_type': 'default'  # default가 일반출금
    }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.post(server_url + "/v1/withdraws/coin", params=query, headers=headers)

#     print(res.json())
#     send_amount = res.json()['amount']
#     send_amount = float(send_amount)
#     print(send_amount)
    return res.json()

# 업비트 입금주소 요청
def up_deposit_address_request(send_to_up_optimal_simbol):
    query = {'currency': send_to_up_optimal_simbol, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key,
               'nonce': str(uuid.uuid4()),
               'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.post(server_url + "/v1/deposits/generate_coin_address", params=query,
                        headers=headers)
    up_add = res.json()['deposit_address']
    up_seadd = res.json()['secondary_address']
    return up_add, up_seadd

def Up_Minimum_Withdraw_Size_digit(sym_name): #자릿수
    server_url = 'https://api.upbit.com'
    query = {'currency': sym_name, }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {'access_key': access_key, 'nonce': str(uuid.uuid4()), 'query_hash': query_hash,
               'query_hash_alg': 'SHA512', }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/withdraws/chance", params=query, headers=headers)

    return float(res.json()['withdraw_limit']['fixed']) #이부분 틀려있었음

C:\Users\Happy\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


<class 'pyupbit.exchange_api.Upbit'>
<module 'ccxt' from 'C:\\Users\\Happy\\anaconda3\\lib\\site-packages\\ccxt\\__init__.py'>


In [3]:
# api 키는 바이낸스와 unbit apikey에 저장해둔다
# api 키 값 로드
with open('apikeys.pickle', 'rb') as fr:
    apikeys = pickle.load(fr)

# 암호키 가져옴.
request_client = RequestClient(api_key=g_api_key, secret_key=g_secret_key)
# request_client_D = RequestClient_D(api_key=g_api_key, secret_key=g_secret_key)
server_url = 'https://api.upbit.com'
Usdt = upbit_get_usd_krw()
# Usdt = 1170
print("Usdt =", Usdt)

# 업비트 잔고 조회 및 해당 코인 가격에 맞는 가격 도출 기능
access_key = apikeys['up_access_key']
secret_key = apikeys['up_secret_key']
upbit = pyupbit.Upbit(access_key, secret_key)

# 바이낸스 잔고조회
binance = ccxt.binance({
    'apiKey': g_api_key,
    'secret': g_secret_key})

binance_fu = ccxt.binance({
    'apiKey': g_api_key,
    'secret': g_secret_key,
    'options': {'defaultType': 'future'}})

client_spot = Spot(key=g_api_key
                   , secret=g_secret_key)

client = Client(api_key=g_api_key,
                api_secret=g_secret_key)

def cheak_about_regester_by_upbit():
    with open('regester_by_upbit.pickle', 'rb') as fr:
        regester_by_upbit = pickle.load(fr)
        past_sym_name = regester_by_upbit["past_sym_name"]

    sym_name = sym_intersection()
    
    New_sym_bols = list(set(sym_name) - set(regester_by_upbit['past_sym_name']))
    
    if(len(set(sym_name) - set(regester_by_upbit['past_sym_name'])) != 0):
        print("==================================warning==================================")
        print("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)",list(set(sym_name) - set(regester_by_upbit['past_sym_name'])))
        print("==================================warning==================================")
        
        print("새로운 출금 주소를 등록하시고 진행하시겠습니까 ? (Y/N)")
        print("Y입력 시 업비트에 출금 주소 입력하고 진행")
        print("N입력 시 새로 추가된 심볼은 사용하지 않고 진행함.")
        input_command  = input()
        
        if (input_command == "Y"):
            import warnings
            warnings.warn("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)", DeprecationWarning)
            sys.exit() ## 강제로 파이썬 종료

        elif(input_command == "N"):
            User_data['ban_list'] = User_data['ban_list'] + New_sym_bols
#             return User_data

        else:
            print("input 입력 오류")
            sys.exit() ## 강제로 파이썬 종료
            
def cheak_about_regester_by_bin():
    with open('regester_by_binance.pickle', 'rb') as fr:
        regester_by_binance = pickle.load(fr)
        past_sym_name = regester_by_binance["past_sym_name"]

    sym_name = sym_intersection()
    
    New_sym_bols = list(set(sym_name) - set(regester_by_binance['past_sym_name']))
    
    if(len(set(sym_name) - set(regester_by_binance['past_sym_name'])) != 0):
        print("==================================warning==================================")
        print("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)",list(set(sym_name) - set(regester_by_binance['past_sym_name'])))
        print("==================================warning==================================")
        
        print("새로운 출금 주소를 등록하시고 진행하시겠습니까 ? (Y/N)")
        print("Y입력 시 업비트에 출금 주소 입력하고 진행")
        print("N입력 시 새로 추가된 심볼은 사용하지 않고 진행함.")
        input_command  = input()
        
        if (input_command == "Y"):
            import warnings
            warnings.warn("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)", DeprecationWarning)
            sys.exit() ## 강제로 파이썬 종료

        elif(input_command == "N"):
            User_data['ban_list'] = User_data['ban_list'] + New_sym_bols
#             return User_data

        else:
            print("input 입력 오류")
            sys.exit() ## 강제로 파이썬 종료
            
def Save_ALL_DATA():
    ## 펑션을 종료시키고 싶을 경우 해당 로직 실행 
    # save data 파이썬 딕셔너리 자료형 데이터 저장.
    with open('Free_Risk_Arbitrage_asset_division.pickle','wb') as fw:
        pickle.dump(asset_division, fw)

    with open('Free_Risk_Arbitrage_User_data.pickle','wb') as fw:
        pickle.dump(User_data, fw)

    for i in range(len(asset_division)):

        print(i ,asset_division[i]['up_asset'] , asset_division[i]['State'])
    print("====================asset_division============================")
    print(asset_division)
    print("=======================User_data==============================")
    print(User_data)
    
def set_premium_asset_division(): # 입력은 %로
    print("%단위로 입력할것 (김프가 그대로)")
    
    print("역프~약김프 최대 수치(프로그램 실행 시), 추천 => 1")
    a_1_1 = float(input())# a_1_1 = 2.5
    print("최소 차익 볼 김프, 추천 => 1.5 ")
    a_2_1 = float(input())# a_2_2 = 1
    print("조정하고 싶은 레버리지 비율 기본 10배")
    adjust_leverage = float(input())
    
    print("역프~약김프 최대 수치(프로그램 실행 시 )",a_1_1,"%")
    print("최소 차익 볼 김프", ((1 + a_2_1 /100) /(1 - a_1_1/100) - 1) * 100,"%")
    
    a_1_1 =  a_1_1 / 100
    a_2_1 =  a_2_1 / 100 
    for i in range(len(asset_division)):
        asset_division[i]['want_premium_step_1_1'] = a_1_1
        asset_division[i]['want_premium_step_2_1'] = a_2_1
        asset_division[i]['adjust_leverage'] = adjust_leverage
        
    return asset_division

def check_logic(asset_division):
    #검산용
    sum_1 = 0
    sum_2 = 0
    for i in range(len(asset_division)):
        sum_1 = sum_1 + asset_division[i]["up_asset"]
        sum_2 = sum_2 + asset_division[i]["binance_asset"] * asset_division[i]['Usdt_first'] * asset_division[i]["future_leverage"]
    print("업비트 = ",sum_1,"바이낸스 = ",sum_2)
    if (sum_1 - 1 < sum_2 < sum_1 + 1 ):# 파이썬이 아주 작은 수를 남겨서
        print("사용중인 자본 맞음")
    else:
        import warnings
        warnings.warn("업비트와 바이낸스자본이 맞지 않은 확인 바람.", DeprecationWarning)
        sys.exit() ## 강제로 파이썬 종료    
        
    State_All = [asset_division[w]["State"] for w in range(len(asset_division))]
    for i in range(len(asset_division)):
        if(asset_division[i]["future_leverage"] >= 20 ) & ((len(set(State_All) & {'None', 'send_bin_to_up'}) == 0)):
            import warnings
            warnings.warn("처음 자본 전송일시 다른파일 사용(레버리지 20 배 이상에 포지션이 없음.)", DeprecationWarning)
            sys.exit() ## 강제로 파이썬 종료    
            
def up_avg_buy_price_info(): # @@ 나중에 이 부분 다 바꾸기
    payload = {
        'access_key': access_key,
        'nonce': str(uuid.uuid4()), }

    jwt_token = jwt.encode(payload, secret_key)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(server_url + "/v1/accounts", headers=headers)
    return res.json()

Usdt = 1442.5


In [4]:
def first_asset_division(div_unit):
    try:  # load data 파이썬 딕셔너리 자료형 데이터 로드
        with open('Free_Risk_Arbitrage_asset_division.pickle', 'rb') as fr:
            asset_division = pickle.load(fr)

        with open('Free_Risk_Arbitrage_User_data.pickle', 'rb') as fr:
            User_data = pickle.load(fr)
            qwer = User_data["qwer"]

            sum_1 = 0
            sum_2 = 0
            for i in range(len(asset_division)):
                sum_1 = sum_1 + asset_division[i]["up_asset"]
                sum_2 = sum_2 + asset_division[i]["binance_asset"] * asset_division[i]["Usdt_first"] * asset_division[i]["future_leverage"]

        # 아직 운용 중인 자본이 있다면 로직을 사용하지 않고 그대로 배출..
        All_States = [asset_division[i]["State"] for i in range(len(asset_division))]  # 모든 States를 가져옮
        if (len(set(All_States) & {'step_1_1', 'send_up_to_bin', 'send_up_to_bin_Completion', 'step_1_2', 'Capital_distribution', 'step_2_1', 'send_bin_to_up', 'send_bin_to_up_Completion', 'step_2_2', 'final_fun'}) != 0):  # 교집합이 0일 경우(모든상태가 저기에 포함되지 않을경우)
            print("한 싸이클이 아직 끝나지 않음.")
            return asset_division, User_data
        else:
            # 옵션 1 오류를 내서 종료 => 이 경우 새로운 엑셀 파일 생성 및 새로운 자본배분
            # 옵션 1 선택. 100 만원 이상 자본 운용 안되게 하기 위함.
            sys.exit()
            # 옵션 2 오류가 아닌 이전 데이터 사용 => 이 경우 그냥 옛날에 나뉜대로 사용 새로운 자본배분 x
#             return asset_division, User_data

    except:

        sum_1 = 0
        sum_2 = 0
        asset_division = {}
        now = datetime.datetime.now()
        qwer = str(now.year) + "_" + str(now.month) + "_" + str(now.day)
        columns_name = ['sicle',
                        'k',
                        'up_asset',
                        'binance_asset',
                        'Using_asset',
                        'step_1_1_up_krw_asset',
                        'step_1_1_bin_fu_usdt_asset',
                        'rest_step_1_1_up_krw_asset',
                        'rest_step_1_1_bin_fu_usdt_asset',
                        'x1',
                        'y1',
                        'Usdt_1',
                        'sym_name_a',
                        'future_leverage',
                        'point_of_buy_UP_ticker_1',
                        'point_of_buy_UP_amount_1',
                        'point_of_buy_bin_fu_ticker_1',
                        'point_of_buy_bin_fu_amount_1',
                        'second_using_asset',
                        'step_1_2_bin_spot_usdt_asset',
                        'step_1_2_bin_fu_usdt_asset',
                        'x2',
                        'y2',
                        'Usdt_2',
                        '업=>바 손해액 ((x2 + y2)*Usdt_2) - ((x1+y1)*Usdt_1)',
                        'sym_name_b',
                        'step_2_1_bin_spot_usdt_asset',
                        'step_2_1_bin_fu_usdt_asset',
                        'rest_step_2_1_bin_spot_usdt_asset',
                        'rest_step_2_1_bin_fu_usdt_asset',
                        'x3',
                        'y3',
                        'Usdt_3',
                        'point_of_buy_bin_spot_ticker_2',
                        'point_of_buy_bin_spot_amount_2',
                        'point_of_buy_bin_fu_ticker_2',
                        'point_of_buy_bin_fu_amount_2',
                        'step_2_2_up_krw_asset',
                        'step_2_2_bin_fu_usdt_asset',
                        'x4',
                        'y4',
                        'Usdt_4',
                        'total_before',
                        'total_after',
                        'date',
                        'want_premium_step_1_1', 
                        'want_premium_step_2_1'
                        ]
        # csv파일로 적기 # newline 설정을 안하면 한줄마다 공백있는 줄이 생긴다.
        with open(qwer + 'Free_Risk_Trading_data.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(columns_name)

    # Using_up_asset의 경우를 제한다.
    Sum_Using_Up_asset = 0
    Sum_Using_Bin_asset = 0
    for i in range(len(asset_division)):  # @@ 이렇게 하는 경우 한번 나뉘면 환율이 바뀌어도 소용없이 계속 나뉜상태,..=>포지션 진입 아닌 경우는 지우는/..?
        if (asset_division[i]['State'] == 'None_position'):
            Sum_Using_Up_asset = Sum_Using_Up_asset + asset_division[i]["up_asset"]
            Sum_Using_Bin_asset = Sum_Using_Bin_asset + asset_division[i]["binance_asset"]

    num = len(asset_division)

    sicle = 0
    save_state = {}  # 사전형태
    save_state[sicle] = []

    first_up_asset = upbit.get_balance("KRW")
    up_asset = first_up_asset - first_up_asset * 0.0005  # 계좌 현금 확인
    up_asset = up_asset - up_asset * 0.0005  # 수수료 보정치(슬리피지)

    first_bin_fu_ass = float(next((item for item in client.futures_account()['assets'] if item['asset'] == 'USDT'), None)['availableBalance'])
    #     first_bin_fu_ass = 133 #@@ 삭제 할것
    #     first_bin_fu_ass = up_asset/Usdt / 5 #@@ 삭제 할것
    bin_fu_ass = first_bin_fu_ass - first_bin_fu_ass * 0.0004
    binance_asset = bin_fu_ass * Usdt

    # @ 수수료들 가져오기 ex)client.get_trade_fee()
    up_market_fee = float(upbit.get_chance("KRW-ETC")["maker_bid_fee"]) # 업비트 코인 현물 시장가 #taker,maker 동일해서 그런지 이것만 존재. @ # 다른 (코인)정보로 받아도 됨
    bin_fees = binance_fu.fees
    bin_spot_market_fee = bin_fees["trading"]['taker'] # 바이낸스 현물 시장가 수수료 #taker는 시장가   # maker 0.1% taker 0.1% # 0.001
    bin_future_market_fee = bin_fees["future"]["trading"]["taker"] # 바이낸스 선물 시장가 수수료 #taker는 시장가 
    bin_Coin_M_future_market_fee = bin_fees["delivery"]["trading"]["taker"] # 바이낸스 선물 coin-m 시장가 수수료 #taker는 시장가 

    asset_for_div_UP = up_asset - Sum_Using_Up_asset
    asset_for_div_bin = (bin_fu_ass - Sum_Using_Bin_asset)
    asset_for_div_bin = asset_for_div_bin * Usdt

    if (asset_for_div_UP > asset_for_div_bin):  # 업비트 자본이 큰 경우
        future_leverage_first = math.ceil(asset_for_div_UP / asset_for_div_bin)
    else:  # 바이낸스 자본이 더 큰경우 # 바이낸스 자본비율이
        future_leverage_first = 1  # 그냥 1해도 됨

    if (asset_for_div_UP > asset_for_div_bin):  # 업비트 자본이 큰 경우
        Decimal_rounding = 0.0000000000001
        future_leverage_by_order_set = ((asset_for_div_UP / asset_for_div_bin) // Decimal_rounding) * Decimal_rounding
    #     future_leverage_by_order_set = (asset_for_div_UP / asset_for_div_bin)

    else:  # 바이낸스 자본이 더 큰경우 # 바이낸스 자본비율이
        future_leverage_by_order_set = 1  # 그냥 1해도 됨

    # asset_for_div_bin = asset_for_div_bin * future_leverage_by_order_set

#     if (15 < future_leverage_by_order_set) or (first_up_asset < 50000) or (first_bin_fu_ass < 130):
#         import warnings
#         print(future_leverage_by_order_set, first_bin_fu_ass)
#         warnings.warn("바이낸스 자본 or 배율이 이상함", DeprecationWarning)
#         sys.exit()  ## 강제로 파이썬 종료

    if (future_leverage_first >= 2):
        #     if (future_leverage_first >= 2 and (future_leverage_first < 10)): # @@ option
        while True:
            if ((asset_for_div_UP >= div_unit) and (asset_for_div_bin * future_leverage_by_order_set >= div_unit)):

                asset_division[num] = {'State': "None_position"}
                asset_division[num]['up_asset'] = div_unit
                asset_division[num]['binance_asset'] = (div_unit / future_leverage_by_order_set) / Usdt

                asset_for_div_UP = asset_for_div_UP - div_unit
                asset_for_div_bin = asset_for_div_bin - div_unit / future_leverage_by_order_set
                future_leverage = future_leverage_by_order_set

                asset_division[num]['future_leverage'] = future_leverage
                asset_division[num]['Usdt_first'] = Usdt  # 자본 나눌 때 달러가치
                asset_division[num]['save_state'] = save_state
                asset_division[num]["qwer"] = qwer
                asset_division[num]["div_unit"] = div_unit
                asset_division[num]['want_premium_step_1_1'] = 0.01
                asset_division[num]['want_premium_step_2_1'] = 0.0125
                asset_division[num]['adjust_leverage'] = 10

                num = num + 1

            elif ((asset_for_div_UP <= div_unit / 10) or (asset_for_div_bin <= div_unit / 10) or (asset_for_div_bin < 50000) or (asset_for_div_UP < 50000)):  # --이하는 투자하지않음(최소주문가능량 고려)
                print("option1", "업비트 남은자본:", asset_for_div_UP, "바이낸스 남은자본:", asset_for_div_bin,"자본배율이 10(숫자만 바꾸면 됨) 배 이상 차이남")
                #             asset_division[num] = {'up_asset' : asset_for_div_UP,"binance_asset" : asset_for_div_bin,"future_leverage":1}
                break
            else:
                asset_division[num] = {'State': "None_position"}

                if (asset_for_div_UP > asset_for_div_bin):  # 업비트 자본이 큰 경우
                    Decimal_rounding = 0.0000000000001
                    future_leverage_by_order_set = ((asset_for_div_UP / asset_for_div_bin) // Decimal_rounding) * Decimal_rounding

                asset_division[num]['up_asset'] = asset_for_div_UP
                asset_division[num]['binance_asset'] = asset_for_div_bin / Usdt

                asset_for_div_UP = asset_for_div_UP - asset_for_div_UP
                asset_for_div_bin = asset_for_div_bin - asset_for_div_bin
                future_leverage = future_leverage_by_order_set

                asset_division[num]['future_leverage'] = future_leverage
                asset_division[num]['Usdt_first'] = Usdt  # 자본 나눌 때 달러가치
                asset_division[num]['save_state'] = save_state
                asset_division[num]["qwer"] = qwer
                asset_division[num]["div_unit"] = div_unit
                asset_division[num]['want_premium_step_1_1'] = 0.01
                asset_division[num]['want_premium_step_2_1'] = 0.0125
                asset_division[num]['adjust_leverage'] = 10

    else:  # 2배 차이 안날시
        while True:
            if ((asset_for_div_UP >= div_unit) and (asset_for_div_bin >= div_unit)):
                asset_division[num] = {'State': "None_position"}
                asset_division[num]['up_asset'] = div_unit
                asset_division[num]['binance_asset'] = (div_unit / future_leverage_first) / Usdt

                asset_for_div_UP = asset_for_div_UP - div_unit
                asset_for_div_bin = asset_for_div_bin - div_unit

                future_leverage = 1

                asset_division[num]['future_leverage'] = future_leverage
                asset_division[num]['Usdt_first'] = Usdt  # 자본 나눌 때 달러가치
                asset_division[num]['save_state'] = save_state
                asset_division[num]["qwer"] = qwer
                asset_division[num]["div_unit"] = div_unit
                asset_division[num]['want_premium_step_1_1'] = 0.01
                asset_division[num]['want_premium_step_2_1'] = 0.0125
                asset_division[num]['adjust_leverage'] = 10

                num = num + 1

            # @@ 레버리지가 너무 크면 안좋을 거 같음 => 나누는 단위에 따라서 최소값 제한함
            elif ((asset_for_div_UP <= div_unit / 5) or (asset_for_div_bin <= div_unit / 5)):  # --이하는 투자하지않음(최소주문가능량 고려)
                print("option2", "업비트 남은자본:", asset_for_div_UP, "바이낸스 남은자본:", asset_for_div_bin)
                break

            else:
                asset_division[num] = {'State': "None_position"}

                if (asset_for_div_UP > asset_for_div_bin):  # 업비트 자본이 큰 경우
                    future_leverage = math.floor(asset_for_div_UP / asset_for_div_bin)
                else:  # 바이낸스 자본이 더 큰경우 # 바이낸스 자본비율이
                    future_leverage = 1  # 그냥 1해도 됨

                if (future_leverage != 1):  # @ 바이낸스 자본과 업비트 자본의 비율이 2배 이상이 될 경우.

                    if (asset_for_div_UP < asset_for_div_bin):  # 바이낸스 자본이 더 큰 경우 @@주로 김프니까(역프는..)이 경우는 그냥 업비트 자본 따라가는게 맞겠지..?
                        asset_Up = asset_for_div_UP  # 변수명이 바뀌어서
                        asset_bin = asset_for_div_UP

                    elif (asset_for_div_bin <= asset_for_div_UP):  # 김프의 경우 2배차이가 나기 시작하면
                        asset_bin = asset_for_div_bin
                        asset_Up = asset_for_div_bin * future_leverage  # @이렇게 하면 업비트 자본이 남고 바이낸스는 전부 쓴다.

                elif (asset_for_div_UP < asset_for_div_bin):  # 작은 자본의 가격을 따라간다.
                    asset_bin = asset_for_div_UP
                    asset_Up = asset_bin
                elif (asset_for_div_bin <= asset_for_div_UP):
                    asset_Up = asset_for_div_bin
                    asset_bin = asset_Up

                asset_division[num]['up_asset'] = asset_Up
                asset_division[num]['binance_asset'] = asset_bin / Usdt

                asset_for_div_UP = asset_for_div_UP - asset_Up
                asset_for_div_bin = asset_for_div_bin - asset_bin

                asset_division[num]['future_leverage'] = future_leverage
                asset_division[num]['Usdt_first'] = Usdt
                asset_division[num]['save_state'] = save_state
                asset_division[num]["qwer"] = qwer
                asset_division[num]["div_unit"] = div_unit
                asset_division[num]['want_premium_step_1_1'] = 0.01
                asset_division[num]['want_premium_step_2_1'] = 0.0125
                asset_division[num]['adjust_leverage'] = 10

    #################################################################################
    # if (future_leverage_first >= 2): # @@ 이 부분 다시 돌리면 이상해짐..
    #
    #     for i in range(len(asset_division)):
    #         sum_1 = sum_1 + asset_division[i]["up_asset"]
    #         sum_2 = sum_2 + asset_division[i]["binance_asset"]
    #     print("업비트 = ", sum_1, "바이낸스 = ", sum_2 * Usdt)
    #
    #     if ((up_asset - sum_1) - 1 < asset_for_div_UP < (up_asset - sum_1) + 1) and ((asset_for_div_bin / Usdt - 1) < bin_fu_ass - sum_2 < (asset_for_div_bin / Usdt + 1)):
    #         print("사용중인 자본 맞음")
    #     else:
    #         import warnings
    #         warnings.warn("업비트와 바이낸스자본이 맞지 않은 확인 바람.", DeprecationWarning)
    #         sys.exit()  ## 강제로 파이썬 종료
    #################################################################################

    check_logic(asset_division) # 로직 체크

    # @@ 1시간 이상 걸리는것들 찾아서 추가해 제거하기
    # 분단위 전송 심볼들
    ban_list = ["WAVES",'OMG', 'XMR', 'MANA', 'BCHA', 'BCH', 'AE', 'SAND', 'USDT', 'BSV', 'BTG', 'SRM', 'CHZ', 'SC', 'ANKR', 'STORJ', 'BCD', 'RVN', 'XTZ', 'NEO', 'ONT', 'ZEC', 'SXP', 'ICX', 'CVC', 'ETH', 'BCN', 'ZRX', 'QTUM', 'KNC', 'LSK', 'ETC', 'ADA', 'ENJ', 'LTC', 'BAT', 'DCR', 'BTC', 'LINK']
    # 내가 직접 지정한 심볼들
    # ban_list = ['CHZ', 'ETH', 'ANKR', 'LINK', 'BAT', 'SXP', 'ETC', 'OMG', 'MANA', 'ZRX', 'ENJ', 'STORJ', 'KNC', 'SRM', 'SRM', 'SAND', 'CVC', 'NEO', 'BCH', 'SC', 'QTUM']
    # 분단위가 아닌 심볼들
    # ban_list = ['ADA','AE','ANKR','BAT','BCD','BCH','BCHA','BCN','BSV','BTC','BTG','BTT','CHZ','CVC','DASH','DCR','DGB','DOGE','ENJ','ETC','ETH','ICX','IOTA','KNC','LINK','LSK','LTC','MANA','NEO','OMG','ONT','QTUM','RVN','SAND','SC','SRM','STORJ','SXP','TRX','XMR','XTZ','ZEC','ZRX']
    # 50분 내외로 전송되는 심볼들
    # ban_list = ['ANKR', 'BAT', 'BCD', 'BCH', 'BCHA', 'BSV', 'BTC', 'BTG', 'CHZ', 'CVC', 'DCR', 'ENJ', 'ETC', 'ETH', 'KNC', 'LINK', 'LTC', 'MANA', 'NEO', 'OMG', 'QTUM', 'RVN', 'SAND', 'SC', 'SRM', 'STORJ', 'SXP', 'XMR', 'ZEC', 'ZRX']
    
    User_data = {}
    #     User_data["asset_division"] = asset_division
    User_data['ban_list'] = ban_list
    User_data["up_market_fee"] = up_market_fee  # 0.05%
    User_data["bin_spot_market_fee"] = bin_spot_market_fee  # maker 0.1% taker 0.1%
    User_data["bin_future_market_fee"] = bin_future_market_fee  # maker 0.02% taker 0.04%
    User_data["bin_Coin_M_future_market_fee"] = bin_Coin_M_future_market_fee  # maker 0.01% taker 0.05%

    User_data["qwer"] = qwer

    return asset_division, User_data

In [5]:
##@ 0.기본셋팅
Usdt = upbit_get_usd_krw()
print('Usdt', Usdt)

balance = binance.fetch_balance()  # free:보유중인 코인, used:거래진행중인 코인 total:전체코인
print("총 자본", upbit.get_balance("KRW") + float(next((item for item in client.futures_account()['assets'] if item['asset'] == 'USDT'), None)['walletBalance']) * Usdt + balance['USDT']['free'] * Usdt)

# 업비트 현물계좌 원화(현금) 확인
# first_up_asset = 1000000# $$@@ 가정한부분 나중에 바꾸기
# first_up_asset = upbit.get_balance("KRW")
# up_asset = first_up_asset - first_up_asset * 0.0005 # @ up_market_fee # 계좌 현금 확인
# up_asset = up_asset - up_asset * 0.0005 # @ up_market_fee # 수수료 보정치(슬리피지)
# print(up_asset, "업비드 자본(수수료 제외)")
up_asset_cheak = upbit.get_balance("KRW")
print(up_asset_cheak, "업비드 자본")

# 바이낸스 선물계좌 달러 확인
# bin_fu_ass = 350# $$@@가정하고 계산하는 부분 나중에 바꾸기.
fu_asset_cheak =float(next((item for item in client.futures_account()['assets'] if item['asset'] == 'USDT'), None)['availableBalance'])
print(fu_asset_cheak, "바이낸스 선물계좌 달러 잔고 확인:") #bin_fu_ass

# first_bin_fu_ass = bin_fu_ass
# bin_fu_ass = bin_fu_ass - bin_fu_ass * 0.0004 # @ bin_future_market_fee
# binance_asset = bin_fu_ass * Usdt
# print(binance_asset, "원 바이낸스 자본(조정 전)")

#@@@ 벤리스트 마지막에 처리하기. 일단 복잡하니까.=> 새로추가된 심볼명들 !!제하기
# ban_list = ['CHZ', 'ETH', 'ANKR', 'LINK', 'BAT', 'SXP', 'ETC', 'OMG', 'MANA', 'ZRX', 'ENJ', 'STORJ', 'KNC', 'SRM', 'SRM', 'SAND', 'CVC', 'NEO', 'BCH', 'SC', 'QTUM']


Usdt 1442.5
총 자본 1753832.755137545
1585634.29667102 업비드 자본
116.60205093 바이낸스 선물계좌 달러 잔고 확인:


In [6]:
def cheak_about_regester_by_upbit():
    with open('regester_by_upbit.pickle', 'rb') as fr:
        regester_by_upbit = pickle.load(fr)
        past_sym_name = regester_by_upbit["past_sym_name"]

    sym_name = sym_intersection()
    
    New_sym_bols = list(set(sym_name) - set(regester_by_upbit['past_sym_name']))
    
    if(len(set(sym_name) - set(regester_by_upbit['past_sym_name'])) != 0):
        print("==================================warning==================================")
        print("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)",list(set(sym_name) - set(regester_by_upbit['past_sym_name'])))
        print("==================================warning==================================")
        
        print("새로운 출금 주소를 등록하시고 진행하시겠습니까 ? (Y/N)")
        print("Y입력 시 업비트에 출금 주소 입력하고 진행")
        print("N입력 시 새로 추가된 심볼은 사용하지 않고 진행함.")
        input_command  = input()
        
        if (input_command == "Y"):
            import warnings
            warnings.warn("새로운 심볼들 !!!!(업비트 출금주소 등록할것!)", DeprecationWarning)
            sys.exit() ## 강제로 파이썬 종료

        elif(input_command == "N"):
            User_data['ban_list'] = User_data['ban_list'] + New_sym_bols
#             return User_data

        else:
            print("input 입력 오류")
            sys.exit() ## 강제로 파이썬 종료

In [7]:
# fu_D_setting() # coin_M레버리지 세팅 @@여기서 오래걸림
asset_division , User_data = first_asset_division(800000)# div_unit = 2000000 #자본을 나눌 단위 (krw) #@@ input()을 추가?

cheak_about_regester_by_upbit() #별말 없으면 그냥 진행
# cheak_about_regester_by_bin()
    
check_logic(asset_division)

option1 업비트 남은자본: 0.0 바이낸스 남은자본: 0.0 자본배율이 10(숫자만 바꾸면 됨) 배 이상 차이남
업비트 =  1584049.0587829233 바이낸스 =  1584049.0587829174
사용중인 자본 맞음
54 54
==================================warning==================================
새로운 심볼들 !!!!(업비트 출금주소 등록할것!) ['SAND', 'ANKR', 'OMG', 'SRM', 'STORJ', 'QTUM', 'STMX', 'CVC', 'BAT', 'SC', 'ENJ', 'MTL', 'MANA', 'SXP', 'LINK', '1INCH', 'CHZ', 'GMT', 'NEO', 'AAVE', 'AXS', 'ZRX', 'KNC', 'AVAX']
==================================warning==================================
새로운 출금 주소를 등록하시고 진행하시겠습니까 ? (Y/N)
Y입력 시 업비트에 출금 주소 입력하고 진행
N입력 시 새로 추가된 심볼은 사용하지 않고 진행함.
N
업비트 =  1584049.0587829233 바이낸스 =  1584049.0587829174
사용중인 자본 맞음


In [8]:
asset_division = set_premium_asset_division() # %수치 변화. #@@ 여기도 레버리지에 따라 최소 차익 볼 김프 수치 조정하기

%단위로 입력할것 (김프가 그대로)
역프~약김프 최대 수치(프로그램 실행 시), 추천 => 1
1
최소 차익 볼 김프, 추천 => 1.5 
1.5
조정하고 싶은 레버리지 비율 기본 10배
10
역프~약김프 최대 수치(프로그램 실행 시 ) 1.0 %
최소 차익 볼 김프 2.525252525252508 %


In [9]:
def sym_intersection_BTC_MARKET():
    ######업비트 코인 심볼명########
    up_sym_BTC = pyupbit.get_tickers(fiat="BTC")
    for i in range(len(up_sym_BTC)):
        # tickers[i].replace('KRW-','')
        up_sym_BTC[i] = up_sym_BTC[i][4:]


    up_sym = pyupbit.get_tickers(fiat="KRW")
    for i in range(len(up_sym)):
        # tickers[i].replace('KRW-','')
        up_sym[i] = up_sym[i][4:]

    #####바이낸스 달러 현물가능 코인 심볼명#USDT만 찾기 #####
    bin_sym_BTC = client.get_symbol_ticker()

    for i in range(len(bin_sym_BTC)):
        if (bin_sym_BTC[i]['symbol'][-3:] == 'BTC'):
            bin_sym_BTC[i] = bin_sym_BTC[i]['symbol'][:-3]
        else:
            bin_sym_BTC[i] = 'remove'
    removeAllOccur(bin_sym_BTC, 'remove')

    #####바이낸스 달러 현물가능 코인 심볼명#USDT만 찾기 #####
    bin_sym = client.get_symbol_ticker()

    for i in range(len(bin_sym)):
        if (bin_sym[i]['symbol'][-4:] == 'USDT'):
            bin_sym[i] = bin_sym[i]['symbol'][:-4]
        else:
            bin_sym[i] = 'remove'
    removeAllOccur(bin_sym, 'remove')

    # 바이낸스 usdt, btc 시장에 있고, 업비트 btc시장에 있는 심볼들.. 문제 => 업비트에 비상장 코인, 바이낸스의 비상장 코인 각각 다름..  
    # 여기까지 했을 떄 up_sym_BTC은 바이낸스에만 있음
    # bin_sym은 바이낸스 모든 마켓
    up_sym_BTC = list(set(up_sym_BTC) - set(up_sym)) # 업비트 btc 시장에만 있는 것.
    bin_sym = list(set(bin_sym_BTC) | set(bin_sym)) # 바이낸스 usdt, btc 시장에 있는 것.(합집합)
    sym_name = list((set(up_sym_BTC) & set(bin_sym)))  # 교집합.. 바이낸스 USDT 시장과 BTC시장에 있는것들 .. => 업비트 btc마켓과 바이낸스 USDT마켓에 존재 (교집합)
    
    return sym_name , list(set(sym_name) - set(bin_sym_BTC))

In [10]:
def delet_None_network_BTC_MARKET(all_coin_info, sym_name):
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):  # 바이낸스에서 전송
        withdrawfee = bin_coin_withdrow_fee_BTC_MARKET(all_coin_info, sym_name[i])
        #         print(sym_name[i],withdrawfee)
        if (withdrawfee == "None_network"):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name

def bin_coin_withdrow_fee_BTC_MARKET(all_coin_info, sym_name):
    try:
        aa = next((item for item in all_coin_info if item['coin'] == sym_name), None)['networkList']
        withdrawfee = float(next((item for item in aa if item['network'] == sym_name), None)['withdrawFee'])
        return withdrawfee
    except:
        withdrawfee = "None_network"
        return withdrawfee
        pass

In [11]:
sym_name_BTC ,sym_name_BTC_only_usdt = sym_intersection_BTC_MARKET()  # 업비트 현물 , 바이낸스 현물 , 바이낸스 선물의 교집합을 가진 심볼명만 가져옴
Usdt = upbit_get_usd_krw()  # 업비트에서 1달러당 환율 가져옴
all_coin_info = client_spot.coin_info()  # 코인 정보를 전부 한번만 가져와서 저장함
sym_name_BTC = delet_None_network_BTC_MARKET(all_coin_info, sym_name_BTC)  # 코인 스스로 전송 불가능 한 코인은 제외(바이낸스 체인 ,bnb만 전송가능한코인)

In [12]:
def del_bin_disable_withdrow_deopsit(all_coin_info, sym_name):
    delet_list = []
    delet_list_append = delet_list.append
    for i in range(len(sym_name)):  # 바이낸스에서 입금 가능 , 전송 불가능
        depositEnable, withdrawEnable = bin_coin_withdrow_deopsit_Enable(all_coin_info, sym_name[i])
        #         print(sym_name[i],depositEnable,withdrawEnable)
        if (depositEnable == False or withdrawEnable == False):
            delet_list_append(sym_name[i])
    name = list(set(sym_name) - set(delet_list))
    return name

In [13]:
sym_name_BTC = del_bin_disable_withdrow_deopsit(all_coin_info, sym_name_BTC)  # 바이낸스 입출금 불가능 심볼 삭제
sym_name_BTC = del_up_disable_deopsit(sym_name_BTC)  # 업비트 입금 불가능 심볼 삭제 # @@ 아예 여기서 심볼명 다 정의해..?
sym_name_BTC = del_up_disable_whthdraw(sym_name_BTC)  # 업비트 출금 불가능 심볼 삭제

FIL 해당코인 업비트에서 바이낸스로 입금 불가상태


# 업 = > 바 서칭

In [14]:
send_money = 100000

In [15]:
# 기본 셋팅
ban_list = User_data["ban_list"]
future_leverage = 1
up_asset = send_money
up_market_fee = 0.0005  # 0.05%
up_market_fee_BTC = 0.0025 # 0.25%
bin_spot_market_fee_bTC =  0.00105 # 0.105 
# https://www.binance.com/en/fee/trading

bin_spot_market_fee = binance.fees['trading']['taker']  # maker 0.1% taker 0.1% # 0.001
bin_future_market_fee = User_data["bin_future_market_fee"]
Measure_Usdt = 1

In [16]:
before_step_1_1_up_krw_asset = upbit.get_balance("KRW")
Usdt = upbit_get_usd_krw()  # 그래서 잘 안변하는 달러는 여기서 갱신

# 함수 정의해도 될듯 업비트 사전 평단가, 수량 가져옴
up_previous_ticker_list = []
up_previous_ticker_list_append = up_previous_ticker_list.append
up_previous_amount_list = []
up_previous_amount_list_append = up_previous_amount_list.append

up_avg_buy_price_info_all = up_avg_buy_price_info()



In [17]:
# for i in range(len(sym_name)):
#     try:
#         up_previous_ticker = float(next((item for item in up_avg_buy_price_info_all if item['currency'] == sym_name[i]), None)['avg_buy_price'])
#         up_previous_amount = float(next((item for item in up_avg_buy_price_info_all if item['currency'] == sym_name[i]), None)["balance"])
#     except:
#         up_previous_ticker = 0
#         up_previous_amount = 0
#     up_previous_ticker_list_append(up_previous_ticker)
#     up_previous_amount_list_append(up_previous_amount)

In [18]:
up_withdrawfee, bin_withdrawfee = up_bin_withdraw_fee(sym_name_BTC, all_coin_info)  # 업비트 전송 수수료 , 바이낸스 전송 수수료
up_Min_Withdraw_Size, bin_Min_Withdraw_Size = up_bin_withdraw_Size(sym_name_BTC, all_coin_info)  # 업비트, 바이낸스 전송최소 가능량
# Bin_Fu_Minimum_Order_Size = Bin_Minimum_FU_Order_Size(sym_name)  # 바이낸스 현물 최소 주문 가능량
Bin_Spot_Minimum_Order_Size = Bin_Minimum_Spot_Order_Size(sym_name_BTC)  # 바이낸스 선물 최소 주문가능량



In [19]:
def up_spot_ask_bid_info_BTC(sym_name):
    # 업비트 매수, 매도 호가창 한번에 조회
    up_sym_names = ["BTC-" + sym_name[i] for i in range(len(sym_name))]
    up_spot_ask_bid = pyupbit.get_orderbook(ticker=up_sym_names)
    return up_sym_names, up_spot_ask_bid







In [20]:
def bin_spot_ask_bid_info_BTC(sym_name):  # 바이낸스 현물 호가창 한번에 조회
    bin_spot_ask_bid = client_spot.book_ticker()
    bin_sym_names = [sym_name[i] + "BTC" for i in range(len(sym_name))]
    
    return bin_sym_names, bin_spot_ask_bid

def bin_ask_info(bin_sym_names, bin_spot_ask_bid):  # 바이낸스 현물 호가창 조회로 매수 호가창 가져옴
    bin_spot_ask_price = []
    bin_spot_ask_size = []
    bin_spot_ask_price_append = bin_spot_ask_price.append
    bin_spot_ask_size_append = bin_spot_ask_size.append
    for i in range(len(bin_sym_names)):
        bin_spot_ask_price_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askPrice']))
        bin_spot_ask_size_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['askQty']))
    #         print(bin_spot_ask_price[i],bin_spot_ask_size[i],bin_sym_names[i])
    return bin_spot_ask_price, bin_spot_ask_size


def bin_bid_info(bin_sym_names, bin_spot_ask_bid):  # 바이낸스 현물 호가창 조회로 매도 호가창 가져옴
    bin_spot_bid_price = []
    bin_spot_bid_size = []
    bin_spot_bid_price_append = bin_spot_bid_price.append
    bin_spot_bid_size_append = bin_spot_bid_size.append
    
    for i in range(len(bin_sym_names)):
        bin_spot_bid_price_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidPrice']))
        bin_spot_bid_size_append(float(next((item for item in bin_spot_ask_bid if item['symbol'] == bin_sym_names[i]), None)['bidQty']))
    #         print(bin_spot_bid_price[i],bin_spot_bid_size[i],bin_sym_names[i])
    return bin_spot_bid_price, bin_spot_bid_size

In [40]:
# 김프 어느정도인지 확인하면서..

up_to_bin_pre_rate = []
up_to_bin_pre_rate_append = up_to_bin_pre_rate.append
bin_to_up_pre_rate = []
bin_to_up_pre_rate_append = bin_to_up_pre_rate.append
up_withdrawfee_by_asset = []
up_withdrawfee_by_asset_append = up_withdrawfee_by_asset.append
bin_withdrawfee_by_asset = []
bin_withdrawfee_by_asset_append = bin_withdrawfee_by_asset.append

# 호가창 정보 가져옴

up_sym_names, up_spot_ask_bid = up_spot_ask_bid_info_BTC(sym_name_BTC)
up_ask_price_BTC, up_ask_size_BTC = up_ask_info(up_sym_names, up_spot_ask_bid)  # 업비트 매수 호가창 1
up_bid_price_BTC, up_bid_size_BTC = up_bid_info(up_sym_names, up_spot_ask_bid)  # 업비트 매도 호가창 4

bin_sym_names, bin_spot_ask_bid = bin_spot_ask_bid_info_BTC(list(set(sym_name_BTC) - set(sym_name_BTC_only_usdt)))
bin_spot_ask_price_BTC, bin_spot_ask_size_BTC = bin_ask_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매수 호가창 3
bin_spot_bid_price_BTC, bin_spot_bid_size_BTC = bin_bid_info(bin_sym_names, bin_spot_ask_bid)  # 바-현 매도 호가창 2

sym_name_BTC_only_usdt_names, bin_spot_ask_bid = bin_spot_ask_bid_info(list(set(sym_name_BTC_only_usdt) - set(sym_name_BTC) | (set(sym_name_BTC) & set(sym_name_BTC_only_usdt)))) # 조심해야함 걸리
bin_spot_ask_price, bin_spot_ask_size = bin_ask_info(sym_name_BTC_only_usdt_names, bin_spot_ask_bid)  # 바-현 매수 호가창 3
bin_spot_bid_price, bin_spot_bid_size = bin_bid_info(sym_name_BTC_only_usdt_names, bin_spot_ask_bid)  # 바-현 매도 호가창 2

aa = next((item for item in bin_spot_ask_bid if item['symbol'] == "BTCUSDT"), None)
btc_ask = float(aa['askPrice'])
btc_bid = float(aa['askPrice'])

bb = pyupbit.get_orderbook(ticker="KRW-BTC")['orderbook_units'][0]
KRW_BTC_ask = float(bb["ask_price"])
KRW_BTC_bid = float(bb["bid_price"])

dic_by_price_info = {}
sym_name_bin_btc_only = list(set(sym_name_BTC) - set(sym_name_BTC_only_usdt))
sym_name_bin_usdt_only = list(set(sym_name_BTC_only_usdt) - set(sym_name_BTC) | (set(sym_name_BTC) & set(sym_name_BTC_only_usdt)))

for i in range(len(sym_name_BTC)):
    if (sym_name_BTC[i] in sym_name_bin_usdt_only):
        j = sym_name_bin_usdt_only.index(sym_name_BTC[i]) # 해당 값의 인덱스 번호를 찾아서 도출
        dic_by_price_info[sym_name_BTC[i]] = {"up_ask_price_BTC" : up_ask_price_BTC[i] * KRW_BTC_ask,
                                                "up_bid_price_BTC" : up_bid_price_BTC[i] * KRW_BTC_bid,
                                                "up_ask_size_BTC" : up_ask_size_BTC[i],
                                                "up_bid_size_BTC" : up_bid_size_BTC[i],
                                                "bin_spot_ask_price" : bin_spot_ask_price[j],
                                                "bin_spot_ask_size" : bin_spot_ask_size[j],
                                                "bin_spot_bid_price" : bin_spot_bid_price[j],
                                                "bin_spot_bid_size" : bin_spot_bid_size[j]
                                             }
    else:
        j = sym_name_bin_btc_only.index(sym_name_BTC[i]) # 해당 값의 인덱스 번호를 찾아서 도출
        dic_by_price_info[sym_name_BTC[i]] = {"up_ask_price_BTC" : up_ask_price_BTC[i] * KRW_BTC_ask,
                                                "up_bid_price_BTC" : up_bid_price_BTC[i] * KRW_BTC_bid,
                                                "up_ask_size_BTC" : up_ask_size_BTC[i],
                                                "up_bid_size_BTC" : up_bid_size_BTC[i],
                                                "bin_spot_ask_price" : bin_spot_ask_price_BTC[j] * btc_ask,  # bin_spot_ask_price_btc 이런형식 ㄱ
                                                "bin_spot_ask_size" : bin_spot_ask_size_BTC[j] , # bin_spot_ask_price_btc 이런형식 ㄱ
                                                "bin_spot_bid_price" : bin_spot_bid_price_BTC[j] * btc_bid,  # bin_spot_ask_price_btc 이런형식 ㄱ
                                                "bin_spot_bid_size" : bin_spot_bid_size_BTC[j]  # bin_spot_ask_price_btc 이런형식 ㄱ
                                                }
dic_by_price_info_key = list(dic_by_price_info.keys())

for i in range(len(dic_by_price_info_key)):
    aa = dic_by_price_info[dic_by_price_info_key[i]]

    # 프리미엄들(역프,김프)
    up_to_bin_pre_rate_append(((aa["bin_spot_bid_price"] * Usdt)/ aa["up_ask_price_BTC"]))  # 역프 or 바낸 전송시 예상 손수익 %(김프시 손해)
    bin_to_up_pre_rate_append((aa["up_bid_price_BTC"] / (aa["bin_spot_ask_price"] * Usdt)))  # 김프 or 업비트 전송시 예상 손수익 %(김프시 수익)

#     # 자본대비 수수료 오직 비율로만 따짐.
#     up_withdrawfee_by_asset_append(up_to_bin_pre_rate[i] - ((up_ask_price[i] * up_withdrawfee[i]) / up_asset))  # 내 자본 대비 전송수수료를 포함하여 가장 좋은 최적의 값 서칭
#     bin_withdrawfee_by_asset_append(bin_to_up_pre_rate[i] - ((bin_spot_ask_price[i] * bin_withdrawfee[i]) / (up_asset)))  # 바이낸스 선물에서 매수 할거니 매도호가창조회

a = up_to_bin_pre_rate.index(max(up_to_bin_pre_rate))  # 리스트에서 가장 큰 값 뽑음(업 => 바 가장 큰 값 인덱스) @@맞지? 현재 김프역프 왔다리 갓다리 하니까
b = bin_to_up_pre_rate.index(max(bin_to_up_pre_rate))  # 리스트에서 가장 큰 값 뽑음(바 => 업 가장 큰 값 인덱스)

aa = dic_by_price_info[dic_by_price_info_key[a]]
step_1_1_Measure_Usdt = ((Measure_Usdt / aa["bin_spot_bid_price"]) * aa["up_ask_price_BTC"]) / Measure_Usdt

aa = dic_by_price_info[dic_by_price_info_key[b]]
step_2_1_Measure_Usdt = ((Measure_Usdt / aa["bin_spot_ask_price"]) * aa["up_bid_price_BTC"]) / Measure_Usdt

if ((step_2_1_Measure_Usdt/step_1_1_Measure_Usdt) > 1) & ((step_1_1_Measure_Usdt/step_2_1_Measure_Usdt) > 1):
    print("==============================================")
    print("if문 통과 업=> 바 전송 시 이득, 바=> 업 전송시 이득.인 상태.")
    print("==============================================")
    
Measure_Usdt= 1
statuses = (
    "김프포함 달러가치 (step_1_1)", step_1_1_Measure_Usdt,
    "김프포함 달러가치 (step_2_1)", step_2_1_Measure_Usdt,
    "비율확인(업=>바 수익)",step_2_1_Measure_Usdt/step_1_1_Measure_Usdt ,
    "비율확인(바=>업 수익)", step_1_1_Measure_Usdt/step_2_1_Measure_Usdt
            )
if (len(statuses) != 0):
    clear_output(wait=True)
    display(statuses)
    
print("추가할 내용 => 바이낸스 btc/usdt 마켓 수수료 계산, 전송 수수료 계산 내용 빠져있음 대충계산중.")
if dic_by_price_info_key[a] in sym_name_bin_usdt_only:
    print("보낼 최적 심볼명" ,dic_by_price_info_key[a])
    print("해당코인은 바이낸스 USDT마켓에서 팔면 됨")
else:
    print("보낼 최적 심볼명" ,dic_by_price_info_key[a])
    print("해당코인은 바이낸스 BTC마켓에서 팔면 됨")
    
if dic_by_price_info_key[b] in sym_name_bin_usdt_only:
    print("바=>업 최적 심볼명" ,dic_by_price_info_key[b])
    print("해당코인은 바이낸스 USDT마켓에서 팔면 됨")
else:
    print("바=>업 최적 심볼명" ,dic_by_price_info_key[b])
    print("해당코인은 바이낸스 BTC마켓에서 팔면 됨")

('김프포함 달러가치 (step_1_1)',
 1443.5401916216747,
 '김프포함 달러가치 (step_2_1)',
 1424.9921497994615,
 '비율확인(업=>바 수익)',
 0.9871510042256764,
 '비율확인(바=>업 수익)',
 1.0130162414051358)

추가할 내용 => 바이낸스 btc/usdt 마켓 수수료 계산, 전송 수수료 계산 내용 빠져있음 대충계산중.
보낼 최적 심볼명 RVN
해당코인은 바이낸스 BTC마켓에서 팔면 됨
바=>업 최적 심볼명 INJ
해당코인은 바이낸스 BTC마켓에서 팔면 됨
